In [1]:
import itertools
import pandas as pd

In [2]:
def get_all_substrings(input_string):
  length = len(input_string)
  return set([input_string[i:j+1] for i in range(length) for j in range(i,length)])

In [3]:
def split_string(a_string):
    return ["x = x.split(\"{}\")".format(sep) for sep in get_all_substrings(a_string)]

In [4]:
split_string("abc d")

['x = x.split("abc ")',
 'x = x.split("bc ")',
 'x = x.split("c ")',
 'x = x.split("ab")',
 'x = x.split("c")',
 'x = x.split("c d")',
 'x = x.split("abc d")',
 'x = x.split(" ")',
 'x = x.split("b")',
 'x = x.split(" d")',
 'x = x.split("d")',
 'x = x.split("abc")',
 'x = x.split("bc")',
 'x = x.split("bc d")',
 'x = x.split("a")']

In [5]:
def get_select_combinations(a_list):
    list_of_list = [list(itertools.combinations(range(len(a_list)), k)) for k in range(1, len(a_list) + 1)]
    select_list = list(itertools.chain.from_iterable(list_of_list))
    func_str_list = ["x = [x[i] for i in {}]".format(selected) for selected in select_list]
    return func_str_list

In [6]:
get_select_combinations(["a", "c", "b"])

['x = [x[i] for i in (0,)]',
 'x = [x[i] for i in (1,)]',
 'x = [x[i] for i in (2,)]',
 'x = [x[i] for i in (0, 1)]',
 'x = [x[i] for i in (0, 2)]',
 'x = [x[i] for i in (1, 2)]',
 'x = [x[i] for i in (0, 1, 2)]']

In [7]:
def get_permutations(a_list):
    permutations_list = list(itertools.permutations(range(len(a_list))))
    func_str_list = ["x = [x[i] for i in {}]".format(permutation) for permutation in permutations_list]
    return func_str_list

In [8]:
get_permutations(["a", "c", "b"])

['x = [x[i] for i in (0, 1, 2)]',
 'x = [x[i] for i in (0, 2, 1)]',
 'x = [x[i] for i in (1, 0, 2)]',
 'x = [x[i] for i in (1, 2, 0)]',
 'x = [x[i] for i in (2, 0, 1)]',
 'x = [x[i] for i in (2, 1, 0)]']

In [9]:
def join_list_to_string(a_list, output_string):
    return ["x = \"{}\".join(x)".format(sep) for sep in get_all_substrings(output_string)]

In [10]:
join_list_to_string(["a", "c", "b"], "abds")

['x = "abds".join(x)',
 'x = "ds".join(x)',
 'x = "s".join(x)',
 'x = "bd".join(x)',
 'x = "ab".join(x)',
 'x = "b".join(x)',
 'x = "d".join(x)',
 'x = "bds".join(x)',
 'x = "abd".join(x)',
 'x = "a".join(x)']

In [11]:
def apply_meta_func(input_obj, func):
    func_list = func(input_obj)
    def apply_func(func):
        d = {"x": input_obj}
        exec(func, d)
        return d["x"]
    obj_list = list(map(apply_func, func_list))
    return func_list, obj_list 

In [12]:
def apply_meta_func_on_group(group, meta_func):
    input_obj = group.iloc[0]["obj"]
    func_str = group.iloc[0]["func_str"]
    func_list, obj_list = apply_meta_func(input_obj, meta_func)
    output_data_frame = pd.DataFrame([obj_list, func_list]).T
    output_data_frame.columns = ["obj", "func_str"]
    func_str += "\n" + output_data_frame["func_str"]
    output_data_frame["func_str"] = func_str    
    return output_data_frame

In [41]:
def apply_func_on_input_output_example(row, input_string, output_string):
    func_str = row
    d = {"x": input_string}
    try:
        exec(func_str, d)
        return d["x"] == output_string
    except:
        return False

In [46]:
def code(input_output_examples_list):
    input_string = input_output_examples_list[0][0]
    output_string = input_output_examples_list[0][1]
    input_obj_list = [input_string]
    input_func_str_list = [""]
    func_data_frame = pd.DataFrame([input_obj_list, input_func_str_list]).T
    func_data_frame.columns = ["obj", "func_str"]
    
    join_list_to_string_with_output = lambda a_list: join_list_to_string(a_list, output_string)
    meta_func_list = [split_string, get_select_combinations, get_permutations, join_list_to_string_with_output]
    
    for meta_func in meta_func_list:
        func_data_frame = func_data_frame.groupby(["func_str"]).apply(apply_meta_func_on_group, meta_func)
        func_data_frame.reset_index(drop=True, inplace=True)
        
    print("total number of functions: {}".format(len(func_data_frame)))
        
    possible_code_series = func_data_frame[func_data_frame.obj == output_string]["func_str"]
    
    for input_output_example in input_output_examples_list[1:]:
        is_good_func_series = possible_code_series.apply(apply_func_on_input_output_example, args=(input_output_example))
        possible_code_series = possible_code_series[is_good_func_series]
        
    return possible_code_series

In [47]:
input_output_examples_list = [("aa bb cc", "bbmmccmmaa"), ("ll dd ff", "ddmmffmmll")]
possible_code_series = code(input_output_examples_list)
possible_code_series.apply(print)

total number of functions: 8256

x = x.split(" ")
x = [x[i] for i in (0, 1, 2)]
x = [x[i] for i in (1, 2, 0)]
x = "mm".join(x)


286    None
Name: func_str, dtype: object